In [ ]:
import argilla as rg
import sqlite3
import os
import numpy
import array
import json

In [ ]:
api_url = os.getenv("ARGILLA_API_URL", "http://localhost:6900")
api_key = os.getenv("ARGILLA_API_KEY", "argilla.apikey")

client = rg.Argilla(
    api_url=api_url,
    api_key=api_key
)

In [40]:
dataset = client.datasets("homefitness_dataset_v1")
if dataset is  None:
    settings = rg.Settings(
        guidelines="Classify the video  as fitness related or not.",
        fields=[
            rg.TextField(
                name="Title",
                title="Title",
                use_markdown=False,
            ),
            rg.TextField(
                name="Description",
                title="Description",
                use_markdown=False,
            ),
            rg.TextField(
                name="video_id",
                title="video_id",
                use_markdown=True,
            ),
            rg.TextField(
                name="channel",
                title="channeltitle",
                use_markdown=False,
            ),
             rg.TextField(
                name="view_count",
                title="view_count",
                use_markdown=False,
            ),
            
        ],
        questions=[
            rg.LabelQuestion(
                name="fit_label",
                title="Does this video belong to fitness or not?",
                labels=["Yes", "No"],
            )
        ],
        vectors=[rg.VectorField(
        name="title_desc_vector",
        title="Title Description Vector",
        dimensions=768
                )  ]
    )
    dataset = rg.Dataset(
        name=f"homefitness_dataset_v1",
        workspace="homeworkouts",
        settings=settings,
        client=client,
    )
    dataset.create()


/Users/kanantharaman/anaconda3/lib/python3.11/site-packages/argilla/client.py:338: UserWarning: Dataset with name 'homefitness_dataset_v1' not found in workspace 'homeworkouts'
  warnings.warn(f"Dataset with name {name!r} not found in workspace {workspace.name!r}")


In [ ]:
def dbconnection():
    # Connect to a local SQLite database file
    conn = sqlite3.connect('fitlabeler.db')
    return conn

In [ ]:
def getvideos(connection):
    cursor = connection.cursor()
    cursor.execute("""
        SELECT json_object(
            'video_id', videoid,
            'view_count', COALESCE(view_count, 0),
            'channel', c.channeltitle,
            'title', json_extract(v.payload, '$.title'),
            'description', COALESCE(json_extract(v.payload, '$.description'), ' '),
            'title_desc_vector', embedding
        )
        FROM yt_videos v
        JOIN yt_channels c ON v.channelid = c.channelid
        WHERE v.embedding IS NOT NULL
          AND v.status = 1
          AND json_extract(v.payload, '$.title') IS NOT NULL
        LIMIT 1000
    """)
    videos = cursor.fetchall()
    cursor.close()
    return videos


In [ ]:
def updatevideo(connection, video_id):
    cursor = connection.cursor()
    cursor.execute("update yt_videos set status=2 where videoid=?", video_id=video_id)
    cursor.close()

In [64]:
def checkES(video_id):
    response = dataset.records(query=video_id).to_list(
    flatten=True
    )
    return response

In [ ]:
conn = dbconnection()
i=0
for video in getvideos(conn):
        record = []
        record.append(json.loads(video[0].read()))
        if(not checkES(record[0]['video_id'])):
            i=i+1
            dataset.records.log(record)
            updatevideo(conn,record[0]['video_id'])
        if(i%100==0):
            conn.commit()
conn.commit()
conn.close()
   